In [1]:
from sklearn.datasets import fetch_california_housing

In [2]:
california = fetch_california_housing()
print(california.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

In [3]:
x = california.data
y = california.target #회귀

* 조건 : Functional API
    1. training set / validation set / test set 분리!
    2. hidden layer 최소 3개 사용! (노드 수 자율)
    3. early stopping 적용!(patience 7)
    4. validation_data 옵션에 validation set 적용!

In [4]:
x.shape, y.shape

((20640, 8), (20640,))

In [5]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.2, random_state = 2023)

#validation set은 training set에서 다시 파생된다.
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.2, random_state = 2023)

In [6]:
train_x.shape, train_y.shape, val_x.shape, val_y.shape, test_x.shape, test_y.shape

((13209, 8), (13209,), (3303, 8), (3303,), (4128, 8), (4128,))

In [7]:
import tensorflow as tf
from tensorflow import keras
## 모델링 : Functional API
#1. 세션 클리어
keras.backend.clear_session()

#2. 레이어 사슬처럼 엮기
il = keras.layers.Input(shape = (8, ))
hl = keras.layers.Dense(512, activation = "relu")(il)
hl = keras.layers.Dense(256, activation = "relu")(hl)
hl = keras.layers.Dense(128, activation = "relu")(hl)
ol = keras.layers.Dense(1)(hl)

#3. 모델의 시작/끝 지정
model = keras.models.Model(il, ol)

#4. 컴파일
model.compile(loss = "mse", optimizer = "adam")

#5. 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense (Dense)               (None, 512)               4608      
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 168,961
Trainable params: 168,961
Non-trainable params: 0
_________________________________________________________________


In [8]:
#Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = "val_loss", #관측 대상
                    min_delta = 0,       #관측 대상의 임계값 설정
                    patience = 5,        #몇번 참을레
                    verbose = 1,
                    restore_best_weights = True) #최적 epoch 시점의 가중치로 설정할래?

In [9]:
#val_loss, val_accuracy는 validation set의 loss, accuracy에 적용

In [10]:
#학습
model.fit(train_x, train_y, epochs = 1000, verbose = 1, callbacks = [es], validation_data = (val_x, val_y))
#validation set으로 나누고 스케일링 했으면 validation_split가 아닌 validation_data = ()!!!!!

Epoch 1/1000
413/413 [==============================] - 7s 4ms/step - loss: 128.5626 - val_loss: 7.5460
Epoch 2/1000
413/413 [==============================] - 2s 4ms/step - loss: 8.4587 - val_loss: 1.0125
Epoch 3/1000
413/413 [==============================] - 2s 5ms/step - loss: 2.9594 - val_loss: 0.9219
Epoch 4/1000
413/413 [==============================] - 2s 4ms/step - loss: 2.7995 - val_loss: 1.5378
Epoch 5/1000
413/413 [==============================] - 2s 4ms/step - loss: 8.6610 - val_loss: 2.5872
Epoch 6/1000
413/413 [==============================] - 2s 4ms/step - loss: 2.4889 - val_loss: 0.8678
Epoch 7/1000
413/413 [==============================] - 2s 4ms/step - loss: 1.4586 - val_loss: 0.9438
Epoch 8/1000
413/413 [==============================] - 2s 4ms/step - loss: 2.0143 - val_loss: 0.8474
Epoch 9/1000
413/413 [==============================] - 1s 3ms/step - loss: 1.4763 - val_loss: 0.7166
Epoch 10/1000
413/413 [==============================] - 1s 4ms/step - loss: 0.8

In [14]:
y_pred = model.predict(test_x)
print(y_pred[:50].reshape(-1))
print(test_y[:10])

129/129 [==============================] - 0s 2ms/step
[1.7467085  2.3964748  1.5111096  0.94857043 1.7199929  2.115139
 1.7856045  2.1116292  1.3492928  2.2288797  1.433327   1.5501201
 4.2367835  0.95303375 1.4813766  2.5266633  1.8183097  1.1743228
 1.532265   1.2789762  1.6084864  2.9248314  1.5885389  4.956248
 2.7814856  2.3827689  2.9502175  1.81194    1.9620855  1.5303161
 1.1094644  3.617728   2.553042   1.6721237  1.7684052  2.3270085
 1.5862675  2.4629748  2.0802     2.056177   3.1369603  4.1160316
 0.86803293 1.1872284  1.0205183  2.8803685  2.5475576  0.92691016
 1.2063432  1.7547896 ]
[3.656 2.01  2.132 0.986 1.53  1.326 2.125 3.068 1.573 1.766]
